In [1]:
import logging
from pathlib import Path
import re
from typing import Dict, Iterable, Union

import pandas as pd

csv_path = '../../Data/senseurcity_data_v02/dataset/'
print(list(Path(csv_path).glob('*.csv')))
csv_files = list(
    filter(
        lambda x: re.match(
            r'Antwerp_.*\.csv|Oslo_.*\.csv|Zagreb_.*\.csv',
            x.parts[-1]
        ),
        Path(csv_path).glob('*.csv')
    )
)
logging.info(f'{len(csv_files)} csv files found in {csv_path}')

[PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_653257.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_4047E0.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_40499C.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/ANT_REF_R801_Fidas_UTC.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_40642E.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_406246.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64FD0A.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64B082.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Zagreb_427906.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_651EF5.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Zagreb_406414.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_406424.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64CB78.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_404

In [2]:
class SensEURCity:
    """
    Parent class of both the `LowCostSensor` and `ReferenceMonitor` class.
    """

    def __init__(self, path: Union[Path, Iterable[Path]]):
        """
        Initialises the class. Should not be called directly, only called
        as part of `LowCostSensor` or `ReferenceMonitor` __init__ methods.

        Parameters
        ----------
        path : Path, iterable of Path
            Paths to the csv files
        """
        self.dfs: Dict[str, pd.DataFrame] = dict()
        """
        Processed csvs stored as pandas DataFrames
        """
        self.paths: Iterable[Path] = list()
        """
        Paths to the csv files that are to be processed
        """
        if isinstance(path, Path):
            self.paths: Iterable[Path] = [path]
        else:
            self.paths: Iterable[Path] = path

    def return_dfs(self) -> Dict[str, pd.DataFrame]:
        """
        Returns the dataframes in a dictionary, keys are the device names

        Returns
        -------
        Dict[str, pd.DataFrames]
            Parsed measurement data
        """
        return self.dfs


class LowCostSensor(SensEURCity):
    """
    Used to extract the low-cost sensor data from the SensEURCity project
    """

    def __init__(self, path: Union[Path, Iterable[Path]]):
        """
        Initialises the class

        Parameters
        ----------
        path : Path, iterable of Path
            Paths to the csv files
        """
        super().__init__(path)

    def parse_files(self):
        """
        Parses individual csv files, removing all measurements from collocated
        reference monitors
        """
        for csv in self.paths:
            name = csv.parts[-1][:-4]
            csv_raw = pd.read_csv(csv, low_memory=False)
            sensor = csv_raw.loc[:, filter(lambda x: 'Ref.' not in x, csv_raw.columns)]
            sensor['date'] = pd.to_datetime(sensor['date'])
            sensor = sensor.set_index('date')
            sensor['Location.ID'] = sensor['Location.ID'].fillna('Field')
            object_cols = [col[0] for col in sensor.items() if col[1].dtype == 'object']
            sensor[object_cols] = sensor[object_cols].fillna('None')
            self.dfs[name] = sensor


class ReferenceMonitor(SensEURCity):
    """
    Used to extract reference monitor data from the SensEURCity project
    """

    def __init__(self, path: Union[Path, Iterable[Path]]):
        """
        Initialises the class

        Parameters
        ----------
        path : Path, iterable of Path
            Paths to the csv files
        """
        super().__init__(path)

    def parse_files(self):
        """
        Reads through all csv files to extract reference measurements and
        concatenates, removing duplicates
        """
        for csv in self.paths:
            csv_raw = pd.read_csv(csv, low_memory=False)
            ref = csv_raw.loc[:, ['date', 'Location.ID', *filter(lambda x: 'Ref.' in x, csv_raw.columns)]]
            all_na = ref[filter(lambda x: 'Ref.' in x, ref.columns)].isna().all(axis=1)
            ref = ref[~all_na]
            ref['date'] = pd.to_datetime(ref['date'])
            ref = ref.set_index('date', drop=False)
            ref.columns = [re.sub('^Ref\.', '', i) for i in ref.columns]
            for name, data in ref.groupby('Location.ID'):
                data = data.drop(columns='Location.ID').dropna(axis=1, how='all')
                site = name
                if self.dfs.get(site) is None:
                    self.dfs[site] = data.drop(columns='date')
                else:
                    self.dfs[site] = pd.concat([self.dfs[site], data]).drop_duplicates().sort_index().drop(columns='date')




In [3]:

a = ReferenceMonitor(csv_files)
a.parse_files()

for site, data in a.return_dfs().items():
    print(site)
    display(data)

[PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_653257.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_4047E0.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_40499C.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_40642E.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_406246.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64FD0A.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64B082.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Zagreb_427906.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_651EF5.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Zagreb_406414.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_406424.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Oslo_64CB78.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_4047E7.csv'), PosixPath('../../Data/senseurcity_data_v02/dataset/Antwerp_4043AE.csv'

,Long,Lat,Press,RH,Temp,CO_ppm,NO,NO2,O3,PM1.Fidas,PM2.5.Fidas,PM4.Fidas,PM10.Fidas,PMtot.Fidas,PM10.TEOM,PM2.5.TEOM,PM10,PM2.5,PM1
date,,,,,,,,,,,,,,,,,,,
2020-03-19 12:49:00+00:00,10.724545,59.932301,NaN,NaN,NaN,0.152022,2.434313,5.915680,NaN,NaN,NaN,NaN,NaN,NaN,36.283692,1.917133,36.283692,1.917133,NaN
2020-03-19 13:00:00+00:00,10.724545,59.932301,NaN,NaN,NaN,0.152022,2.434313,5.915680,NaN,NaN,NaN,NaN,NaN,NaN,36.283692,1.917133,NaN,NaN,NaN
2020-03-19 13:01:00+00:00,10.724545,59.932301,NaN,NaN,NaN,0.179328,5.927735,7.244449,NaN,NaN,NaN,NaN,NaN,NaN,46.327092,17.389704,46.327092,17.389704,NaN
2020-03-19 14:01:00+00:00,10.724545,59.932301,NaN,NaN,NaN,0.176065,2.654614,5.155436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-19 15:01:00+00:00,10.724545,59.932301,NaN,NaN,NaN,0.196855,2.844313,7.953421,NaN,NaN,NaN,NaN,NaN,NaN,13.448622,-4.769274,13.448622,-4.769274,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09 06:57:00+00:00,10.724545,59.932301,985.6,77.576937,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-09 06:58:00+00:00,10.724545,59.932301,985.6,77.576937,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-09 06:59:00+00:00,10.724545,59.932301,985.6,77.576937,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ANT_REF_R801


,Long,Lat,Press,RH,Temp,CO_ppm,CO2,NO,NO2,O3,PM10,PM2.5,PM1
date,,,,,,,,,,,,,
2020-02-19 11:50:00+00:00,4.44202,51.20624,1020.2,65.87,7.33,NaN,NaN,2.72,8.376963,26.292683,13.1,6.4,3.1
2020-02-19 11:51:00+00:00,4.44202,51.20624,1020.2,65.44,7.28,NaN,NaN,2.96,8.272251,26.243902,12.9,6.5,3.2
2020-02-19 11:52:00+00:00,4.44202,51.20624,1020.2,65.09,7.32,NaN,NaN,2.80,8.010471,26.780488,12.6,6.6,3.2
2020-02-19 11:53:00+00:00,4.44202,51.20624,1020.2,64.76,7.40,NaN,NaN,2.56,7.068063,27.707317,12.9,6.7,3.3
2020-02-19 11:54:00+00:00,4.44202,51.20624,1020.2,64.15,7.47,NaN,NaN,2.56,6.963351,27.707317,13.8,6.9,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-15 23:57:00+00:00,4.43180,51.20970,1028.9,65.92,NaN,0.167164,NaN,0.16,7.905759,26.585366,15.7,9.3,7.5
2021-04-15 23:58:00+00:00,4.43180,51.20970,1028.9,65.87,NaN,0.176866,NaN,0.24,7.748691,26.878049,15.1,9.3,7.5
2021-04-15 23:58:00+00:00,4.43180,51.20970,1028.9,65.87,NaN,0.176866,NaN,0.24,7.748691,26.878049,15.1,9.3,7.5


ANT_TRA_POTH


...

,Long,Lat,NO,NO2,PM10.TEOM,PM2.5.TEOM,PM10,PM2.5
date,,,,,,,,
2020-11-04 09:00:00+00:00,10.703992,59.911262,113.380977,34.554874,110.088608,14.259615,110.088608,14.259615
2020-11-04 09:01:00+00:00,10.703992,59.911262,20.551664,15.084488,35.037975,9.932692,35.037975,9.932692
2020-11-04 09:02:00+00:00,10.703992,59.911262,20.551664,15.084488,35.037975,9.932692,35.037975,9.932692
2020-11-04 09:03:00+00:00,10.703992,59.911262,20.551664,15.084488,35.037975,9.932692,35.037975,9.932692
2020-11-04 09:04:00+00:00,10.703992,59.911262,20.551664,15.084488,35.037975,9.932692,35.037975,9.932692
...,...,...,...,...,...,...,...,...
2021-03-10 09:56:00+00:00,10.703992,59.911262,24.644716,22.595633,58.708861,5.317308,58.708861,5.317308
2021-03-10 09:57:00+00:00,10.703992,59.911262,24.644716,22.595633,58.708861,5.317308,58.708861,5.317308
2021-03-10 09:58:00+00:00,10.703992,59.911262,24.644716,22.595633,58.708861,5.317308,58.708861,5.317308


ANT_REF_M802


,Long,Lat,RH,Temp,NO,NO2,PM10,PM2.5,PM1
date,,,,,,,,,
2020-06-18 00:00:00+00:00,4.42442,51.26097,98.0,16.1,0.0,11.256545,19.6,16.0,15.5
2020-06-18 00:01:00+00:00,4.42442,51.26097,97.5,15.7,0.0,5.759162,18.1,14.5,13.5
2020-06-18 00:02:00+00:00,4.42442,51.26097,97.5,15.7,0.0,5.759162,18.1,14.5,13.5
2020-06-18 00:03:00+00:00,4.42442,51.26097,97.5,15.7,0.0,5.759162,18.1,14.5,13.5
2020-06-18 00:04:00+00:00,4.42442,51.26097,97.5,15.7,0.0,5.759162,18.1,14.5,13.5
...,...,...,...,...,...,...,...,...,...
2021-02-17 07:54:00+00:00,4.42442,51.26097,NaN,NaN,2.4,15.706806,18.1,9.0,5.0
2021-02-17 07:55:00+00:00,4.42442,51.26097,NaN,NaN,2.4,15.706806,18.1,9.0,5.0
2021-02-17 07:56:00+00:00,4.42442,51.26097,NaN,NaN,2.4,15.706806,18.1,9.0,5.0


OSL_REF_SMS


,Long,Lat,NO,NO2,PM10.TEOM,PM2.5.TEOM,PM10,PM2.5
date,,,,,,,,
2020-11-19 14:00:00+00:00,10.689714,59.919754,17.540826,14.421085,24.629554,2.008703,24.629554,2.008703
2020-11-19 14:01:00+00:00,10.689714,59.919754,12.076844,10.399726,7.653372,-1.386737,7.653372,-1.386737
2020-11-19 14:02:00+00:00,10.689714,59.919754,12.076844,10.399726,7.653372,-1.386737,7.653372,-1.386737
2020-11-19 14:03:00+00:00,10.689714,59.919754,12.076844,10.399726,7.653372,-1.386737,7.653372,-1.386737
2020-11-19 14:04:00+00:00,10.689714,59.919754,12.076844,10.399726,7.653372,-1.386737,7.653372,-1.386737
...,...,...,...,...,...,...,...,...
2021-03-10 07:56:00+00:00,10.689714,59.919754,36.323873,31.567362,9.919172,3.879662,NaN,NaN
2021-03-10 07:57:00+00:00,10.689714,59.919754,36.323873,31.567362,9.919172,3.879662,NaN,NaN
2021-03-10 07:58:00+00:00,10.689714,59.919754,36.323873,31.567362,9.919172,3.879662,NaN,NaN


In [3]:
for csv in csv_files:
    a = LowCostSensor(csv)
    a.parse_files()
    for site, data in a.return_dfs().items():
        print(site)
        display(data)

Oslo_653257


,latitude,longitude,altitude,Location.ID,BMP280,SHT31HE,SHT31TE,Absolute_humidity,Td_deficit,SHT31HI,...,5310CST_flag,OPCN3PM10_flag,5325CAT_flag,5325CST_flag,OPCN3PM25_flag,BMP280_flag,SHT31HE_flag,SHT31HI_flag,SHT31TE_flag,SHT31TI_flag
date,,,,,,,,,,,,,,,,,,,,,
2020-09-09 13:14:00+00:00,59.932308,10.724497,59.486111,OSL_REF_KVN,995.916667,15.914651,76.735078,NaN,NaN,16.547519,...,W,W,W,W,W,W,W,W,W,W
2020-09-09 13:15:00+00:00,59.932320,10.724507,59.520000,OSL_REF_KVN,995.958333,15.482948,75.976196,NaN,NaN,15.851453,...,W,W,W,W,W,W,W,W,W,W
2020-09-09 13:16:00+00:00,59.932368,10.724549,60.285714,OSL_REF_KVN,995.958333,15.565728,75.416190,NaN,NaN,16.086442,...,W,W,W,W,W,W,W,W,W,W
2020-09-09 13:17:00+00:00,59.932372,10.724556,60.428571,OSL_REF_KVN,995.916667,15.613794,74.927901,NaN,NaN,16.257343,...,W,W,W,W,W,W,W,W,W,W
2020-09-09 13:18:00+00:00,59.932378,10.724566,60.340000,OSL_REF_KVN,NaN,NaN,NaN,NaN,NaN,NaN,...,W,W,W,W,W,W,W,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09 05:56:00+00:00,59.932303,10.724468,67.214286,OSL_REF_KVN,980.125000,5.343709,71.801328,4.991509,4.738112,7.824445,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 05:57:00+00:00,59.932303,10.724470,67.071429,OSL_REF_KVN,980.083333,5.343709,71.929503,5.000419,4.713124,7.824445,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 05:58:00+00:00,59.932302,10.724473,66.642857,OSL_REF_KVN,980.062500,5.346380,71.863890,4.996737,4.726008,7.827115,...,None,None,None,None,None,None,None,None,None,None


Antwerp_4047E0


,latitude,longitude,altitude,Location.ID,BMP280,SHT31HE,SHT31TE,Absolute_humidity,Td_deficit,SHT31HI,...,OPCN3PM10_flag,5310CST_flag,5325CAT_flag,5325CST_flag,OPCN3PM25_flag,BMP280_flag,SHT31HE_flag,SHT31HI_flag,SHT31TE_flag,SHT31TI_flag
date,,,,,,,,,,,,,,,,,,,,,
2020-04-02 10:37:00+00:00,51.209663,4.431819,8.854545,ANT_REF_R801,1015.010417,12.856680,40.717174,NaN,NaN,11.530861,...,W,W,W,W,W,W,W,W,W,W
2020-04-02 10:38:00+00:00,51.209661,4.431836,5.721212,ANT_REF_R801,1014.854167,11.995499,40.866712,NaN,NaN,10.059510,...,W,W,W,W,W,W,W,W,W,W
2020-04-02 10:39:00+00:00,51.209663,4.431860,4.114286,ANT_REF_R801,1014.958333,11.138323,42.273594,NaN,NaN,10.281147,...,W,W,W,W,W,W,W,W,W,W
2020-04-02 10:40:00+00:00,51.209667,4.431837,3.033333,ANT_REF_R801,1014.937500,10.751125,43.697261,NaN,NaN,10.329213,...,W,W,W,W,W,W,W,W,W,W
2020-04-02 10:41:00+00:00,51.209664,4.431850,1.771429,ANT_REF_R801,1014.958333,10.743114,44.299992,NaN,NaN,10.382620,...,W,W,W,W,W,W,W,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-13 07:57:00+00:00,51.209656,4.431861,8.600000,ANT_REF_R801,1033.875000,7.698939,60.756847,4.925417,7.181669,9.797818,...,None,None,None,None,None,None,None,None,None,None
2021-04-13 07:58:00+00:00,51.209655,4.431864,8.600000,ANT_REF_R801,1033.854167,7.712291,60.608835,4.917660,7.216463,9.821851,...,None,None,None,None,None,None,None,None,None,None
2021-04-13 07:59:00+00:00,51.209655,4.431867,8.600000,ANT_REF_R801,1033.895833,7.733654,60.436408,4.910443,7.257417,9.845884,...,None,None,None,None,None,None,None,None,None,None


Antwerp_40499C


...

,latitude,longitude,altitude,Location.ID,BMP280,SHT31HE,SHT31TE,Absolute_humidity,Td_deficit,SHT31HI,...,5310CST_flag,OPCN3PM10_flag,5325CAT_flag,5325CST_flag,OPCN3PM25_flag,BMP280_flag,SHT31HE_flag,SHT31HI_flag,SHT31TE_flag,SHT31TI_flag
date,,,,,,,,,,,,,,,,,,,,,
2020-09-11 09:08:00+00:00,59.932280,10.724544,81.000000,OSL_REF_KVN,1003.114583,14.218547,56.458381,NaN,NaN,15.842107,...,W,W,W,W,W,W,W,W,W,W
2020-09-11 09:09:00+00:00,59.932282,10.724542,80.884615,OSL_REF_KVN,1002.937500,13.482872,57.311360,NaN,NaN,14.390784,...,W,W,W,W,W,W,W,W,W,W
2020-09-11 09:10:00+00:00,59.932295,10.724539,79.971429,OSL_REF_KVN,1003.062500,13.496223,58.043793,NaN,NaN,14.668498,...,W,W,W,W,W,W,W,W,W,W
2020-09-11 09:11:00+00:00,59.932316,10.724528,78.985714,OSL_REF_KVN,1003.083333,13.533608,58.329137,NaN,NaN,14.855421,...,W,W,W,W,W,W,W,W,W,W
2020-09-11 09:12:00+00:00,59.932329,10.724509,77.737500,OSL_REF_KVN,1003.041667,13.570993,58.367285,NaN,NaN,14.996948,...,W,W,W,W,W,W,W,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09 06:56:00+00:00,59.932250,10.724474,77.975000,OSL_REF_KVN,978.729167,5.613413,67.891966,4.804268,5.531136,8.211643,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 06:57:00+00:00,59.932254,10.724473,78.050000,OSL_REF_KVN,978.729167,5.605402,68.134585,4.818898,5.481060,8.206302,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 06:58:00+00:00,59.932258,10.724472,78.066667,OSL_REF_KVN,978.708333,5.613413,68.288701,4.832342,5.449883,8.203632,...,None,None,None,None,None,None,None,None,None,None


Oslo_652D3A


,latitude,longitude,altitude,Location.ID,BMP280,SHT31HE,SHT31TE,Absolute_humidity,Td_deficit,SHT31HI,...,5310CST_flag,OPCN3PM10_flag,5325CAT_flag,5325CST_flag,OPCN3PM25_flag,BMP280_flag,SHT31HE_flag,SHT31HI_flag,SHT31TE_flag,SHT31TI_flag
date,,,,,,,,,,,,,,,,,,,,,
2020-08-28 08:23:00+00:00,59.932387,10.724603,74.200000,OSL_REF_KVN,1002.937500,16.818112,64.528878,NaN,NaN,18.090524,...,W,W,W,W,W,W,W,W,W,W
2020-08-28 08:24:00+00:00,59.932387,10.724603,74.200000,OSL_REF_KVN,1002.812500,16.075761,65.145342,NaN,NaN,16.657893,...,W,W,W,W,W,W,W,W,W,W
2020-08-28 08:25:00+00:00,59.932387,10.724604,74.200000,OSL_REF_KVN,1002.958333,16.137179,65.404746,NaN,NaN,16.938277,...,W,W,W,W,W,W,W,W,W,W
2020-08-28 08:26:00+00:00,59.932387,10.724605,74.200000,OSL_REF_KVN,1002.979167,16.187915,65.374228,NaN,NaN,17.127871,...,W,W,W,W,W,W,W,W,W,W
2020-08-28 08:27:00+00:00,59.932385,10.724607,74.283333,OSL_REF_KVN,1002.958333,16.273365,65.322347,NaN,NaN,17.277409,...,W,W,W,W,W,W,W,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09 06:58:00+00:00,59.932248,10.724536,70.300000,OSL_REF_KVN,979.083333,5.546654,69.480430,4.895135,5.205598,7.800412,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 06:59:00+00:00,59.932251,10.724555,70.342857,OSL_REF_KVN,979.041667,5.554665,69.286641,4.884055,5.244942,7.805753,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 07:00:00+00:00,59.932253,10.724557,70.400000,OSL_REF_KVN,979.020833,5.565347,69.436179,4.898036,5.215249,7.821775,...,None,None,None,None,None,None,None,None,None,None


Oslo_647D5A


,latitude,longitude,altitude,Location.ID,BMP280,SHT31HE,SHT31TE,Absolute_humidity,Td_deficit,SHT31HI,...,5310CST_flag,OPCN3PM10_flag,5325CAT_flag,5325CST_flag,OPCN3PM25_flag,BMP280_flag,SHT31HE_flag,SHT31HI_flag,SHT31TE_flag,SHT31TI_flag
date,,,,,,,,,,,,,,,,,,,,,
2020-09-10 11:34:00+00:00,59.932332,10.724468,62.800,OSL_REF_KVN,1006.291667,17.225338,33.233387,NaN,NaN,19.177348,...,W,W,W,W,W,W,W,W,W,W
2020-09-10 11:35:00+00:00,59.932332,10.724468,62.775,OSL_REF_KVN,1006.145833,16.532387,33.633936,NaN,NaN,17.744717,...,W,W,W,W,W,W,W,W,W,W
2020-09-10 11:36:00+00:00,59.932332,10.724468,62.700,OSL_REF_KVN,1006.291667,16.559091,33.632410,NaN,NaN,17.977035,...,W,W,W,W,W,W,W,W,W,W
2020-09-10 11:37:00+00:00,59.932332,10.724467,62.700,OSL_REF_KVN,1006.312500,16.593805,33.289082,NaN,NaN,18.131914,...,W,W,W,W,W,W,W,W,W,W
2020-09-10 11:38:00+00:00,59.932332,10.724466,62.700,OSL_REF_KVN,1006.333333,16.628519,33.388266,NaN,NaN,18.230716,...,W,W,W,W,W,W,W,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09 06:56:00+00:00,59.932340,10.724511,68.000,OSL_REF_KVN,978.916667,5.624094,69.121843,4.894734,5.281021,8.160906,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 06:57:00+00:00,59.932340,10.724512,67.900,OSL_REF_KVN,978.895833,5.656138,69.196612,4.910361,5.267211,8.155566,...,None,None,None,None,None,None,None,None,None,None
2021-04-09 06:58:00+00:00,59.932339,10.724513,67.900,OSL_REF_KVN,978.854167,5.680171,69.373617,4.930702,5.232462,8.163577,...,None,None,None,None,None,None,None,None,None,None
